In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Hello world

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Versiones de paquetes</b></summary>

El código en esta página se desarrolló utilizando los siguientes requisitos.
Recomendamos usar estas versiones o versiones más recientes.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
Este ejemplo contiene dos partes. Primero creará un programa cuántico simple y lo ejecutará en una unidad de procesamiento cuántico (QPU). Debido a que la investigación cuántica real requiere programas mucho más robustos, en la segunda sección ([Escalar a grandes números de qubits](#scale-to-large-numbers-of-qubits)), escalará el programa simple hasta el nivel de utilidad.
## Instalar y autenticar
1. Si aún no ha instalado Qiskit, encuentre instrucciones en la guía [Inicio rápido](/guides/quick-start).

    - Instale Qiskit Runtime para ejecutar trabajos en hardware cuántico:

        ```bash
        pip install qiskit-ibm-runtime
        ```

    - Configure un entorno para ejecutar notebooks de Jupyter localmente:

        ```bash
        pip install jupyter
        ```

2. Configure su autenticación para acceder al hardware cuántico a través del [Plan Abierto](/guides/plans-overview#open-plan) gratuito.

    (Si recibió una invitación por correo electrónico para unirse a una cuenta, siga los [pasos para usuarios invitados](/guides/cloud-setup-invited) en su lugar.)

    - Vaya a [IBM Quantum Platform](https://quantum.cloud.ibm.com/) para iniciar sesión o crear una cuenta.
         > **Note:** Si se conecta a través de un servidor proxy, debe usar Qiskit Runtime v0.44.0 o posterior.
    - Genere su clave API (también llamada *token API*) en el [panel de control](https://quantum.cloud.ibm.com/), luego cópiela en una ubicación segura.
    - Vaya a la página de [Instancias](https://quantum.cloud.ibm.com/instances) y encuentre la instancia que desea usar. Coloque el cursor sobre su CRN y haga clic para copiarlo.

    - Guarde sus credenciales localmente con este código:

        ```python
        from qiskit_ibm_runtime import QiskitRuntimeService

        QiskitRuntimeService.save_account(
        token="<your-api-key>", # Use la clave API de 44 caracteres que creó y guardó del panel de control de IBM Quantum Platform
        instance="<CRN>", # Opcional
        )
        ```

3. Ahora puede usar este código Python cada vez que desee autenticarse en Qiskit Runtime Service:
    ```python
        from qiskit_ibm_runtime import QiskitRuntimeService

        # Ejecute cada vez que necesite el servicio
        service = QiskitRuntimeService()
    ```
> **Info:** Si está usando una computadora pública u otro entorno no seguro, siga las [instrucciones de autenticación manual](/guides/cloud-setup-untrusted) en su lugar para mantener seguras sus credenciales de autenticación.
## Crear y ejecutar un programa cuántico simple
Los cuatro pasos para escribir un programa cuántico usando patrones de Qiskit son:

1.  Mapear el problema a un formato nativo cuántico.

2.  Optimizar los circuitos y operadores.

3.  Ejecutar usando una función primitiva cuántica.

4.  Analizar los resultados.

### Paso 1. Mapear el problema a un formato nativo cuántico
En un programa cuántico, los *circuitos cuánticos* son el formato nativo para representar instrucciones cuánticas, y los *operadores* representan los observables que se medirán. Al crear un circuito, generalmente creará un nuevo objeto [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class), luego le agregará instrucciones en secuencia.
La siguiente celda de código crea un circuito que produce un *estado de Bell*, que es un estado en el que dos qubits están completamente entrelazados entre sí.

> **Note:** El SDK de Qiskit utiliza la numeración de bits LSb 0 donde el dígito $n^{th}$ tiene el valor $1 \ll n$ o $2^n$. Para más detalles, consulte el tema [Ordenamiento de bits en el SDK de Qiskit](/guides/bit-ordering).

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorOptions
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from matplotlib import pyplot as plt
# Uncomment the next line if you want to use a simulator:
# from qiskit_ibm_runtime.fake_provider import FakeBelemV2


# Create a new circuit with two qubits
qc = QuantumCircuit(2)

# Add a Hadamard gate to qubit 0
qc.h(0)

# Perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)

# Return a drawing of the circuit using MatPlotLib ("mpl").
# These guides are written by using Jupyter notebooks, which
# display the output of the last line of each cell.
# If you're running this in a script, use `print(qc.draw())` to
# print a text drawing.
qc.draw("mpl")

<Image src="../docs/images/guides/hello-world/extracted-outputs/930ca3b6-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/930ca3b6-0.svg)

Consulte [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class) en la documentación para todas las operaciones disponibles.
Al crear circuitos cuánticos, también debe considerar qué tipo de datos desea que se devuelvan después de la ejecución. Qiskit proporciona dos formas de devolver datos: puede obtener una distribución de probabilidad para un conjunto de qubits que elija medir, o puede obtener el valor esperado de un observable. Prepare su carga de trabajo para medir su circuito de una de estas dos formas con [primitivas de Qiskit](/guides/get-started-with-primitives) (explicadas en detalle en el [Paso 3](#step-3-execute-using-the-quantum-primitives)).

Este ejemplo mide valores esperados utilizando el submódulo `qiskit.quantum_info`, que se especifica mediante operadores (objetos matemáticos utilizados para representar una acción o proceso que cambia un estado cuántico). La siguiente celda de código crea seis operadores de Pauli de dos qubits: `IZ`, `IX`, `ZI`, `XI`, `ZZ` y `XX`.

In [2]:
# Set up six different observables.

observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]

> **Note:** Aquí, algo como el operador `ZZ` es una abreviatura del producto tensorial $Z\otimes Z$, lo que significa medir Z en el qubit 1 y Z en el qubit 0 juntos, y obtener información sobre la correlación entre el qubit 1 y el qubit 0. Los valores esperados como este también se escriben típicamente como $\langle Z_1 Z_0 \rangle$.
> 
> Si el estado está entrelazado, entonces la medición de $\langle Z_1 Z_0 \rangle$ debería ser diferente de la medición de $\langle I_1 \otimes Z_0 \rangle \langle Z_1 \otimes I_0 \rangle$. Para el estado entrelazado específico creado por nuestro circuito descrito anteriormente, la medición de $\langle Z_1 Z_0 \rangle$ debería ser 1 y la medición de $\langle I_1 \otimes Z_0 \rangle \langle Z_1 \otimes I_0 \rangle$ debería ser cero.
<span id="optimize"></span>

### Paso 2. Optimizar los circuitos y operadores

Al ejecutar circuitos en un dispositivo, es importante optimizar el conjunto de instrucciones que contiene el circuito y minimizar la profundidad general (aproximadamente el número de instrucciones) del circuito. Esto garantiza que obtenga los mejores resultados posibles al reducir los efectos del error y el ruido. Además, las instrucciones del circuito deben ajustarse a la [Arquitectura del conjunto de instrucciones (ISA)](/guides/transpile#instruction-set-architecture) del dispositivo backend y deben considerar las compuertas base y la conectividad de qubits del dispositivo.

El siguiente código instancia un dispositivo real para enviar un trabajo y transforma el circuito y los observables para que coincidan con la ISA de ese backend. Requiere que ya haya [guardado sus credenciales](/guides/cloud-setup)

In [ ]:
service = QiskitRuntimeService()

backend = service.least_busy(simulator=False, operational=True)

# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

isa_circuit.draw("mpl", idle_wires=False)

<Image src="../docs/images/guides/hello-world/extracted-outputs/9a901271-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/9a901271-0.svg)

### Paso 3. Ejecutar usando las primitivas cuánticas
Las computadoras cuánticas pueden producir resultados aleatorios, por lo que generalmente se recopila una muestra de las salidas ejecutando el circuito muchas veces. Puede estimar el valor del observable usando la clase `Estimator`. `Estimator` es una de dos [primitivas](/guides/get-started-with-primitives); la otra es `Sampler`, que se puede usar para obtener datos de una computadora cuántica. Estos objetos poseen un método `run()` que ejecuta la selección de circuitos, observables y parámetros (si corresponde), usando un [bloque unificado primitivo (PUB).](/guides/primitives#sampler)

In [4]:
# Construct the Estimator instance.

estimator = Estimator(mode=backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 5000

mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

# One pub, with one circuit to run against five different observables.
job = estimator.run([(isa_circuit, mapped_observables)])

# Use the job ID to retrieve your job data later
print(f">>> Job ID: {job.job_id()}")

>>> Job ID: d5k96q4jt3vs73ds5tgg


After a job is submitted, you can wait until either the job is completed within your current python instance, or use the `job_id` to retrieve the data at a later time.  (See the [section on retrieving jobs](/docs/guides/monitor-job#retrieve-job-results-at-a-later-time) for details.)

After the job completes, examine its output through the job's `result()` attribute.

In [5]:
# This is the result of the entire submission.  You submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had six observables,
# so contains information on all six.
pub_result = job.result()[0]

In [6]:
# Check there are six observables.
# If not, edit the comments in the previous cell and update this test.
assert len(pub_result.data.evs) == 6

<Admonition type="note" title="Alternative: run the example using a simulator">
  When you run your quantum program on a real device, your workload must wait in a queue before it runs. To save time, you can instead use the following code to run this small workload on the [`fake_provider`](../api/qiskit-ibm-runtime/fake-provider) with the Qiskit Runtime local testing mode. Note that this is only possible for a small circuit. When you scale up in the next section, you will need to use a real device.

  ```python

  # Use the following code instead if you want to run on a simulator:

  from qiskit_ibm_runtime.fake_provider import FakeBelemV2
  backend = FakeBelemV2()
  estimator = Estimator(backend)

  # Convert to an ISA circuit and layout-mapped observables.

  pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
  isa_circuit = pm.run(qc)
  mapped_observables = [
      observable.apply_layout(isa_circuit.layout) for observable in observables
  ]

  job = estimator.run([(isa_circuit, mapped_observables)])
  result = job.result()

  # This is the result of the entire submission.  You submitted one Pub,
  # so this contains one inner result (and some metadata of its own).

  job_result = job.result()

  # This is the result from our single pub, which had five observables,
  # so contains information on all five.

  pub_result = job.result()[0]
  ```
</Admonition>

### Step 4. Analyze the results

The analyze step is typically where you might post-process your results using, for example, measurement error mitigation or zero noise extrapolation (ZNE). You might feed these results into another workflow for further analysis or prepare a plot of the key values and data. In general, this step is specific to your problem.  For this example, plot each of the expectation values that were measured for our circuit.

The expectation values and standard deviations for the observables you specified to Estimator are accessed through the job result's `PubResult.data.evs` and `PubResult.data.stds` attributes. To obtain the results from Sampler, use the `PubResult.data.meas.get_counts()` function, which will return a `dict` of measurements in the form of bitstrings as keys and counts as their corresponding values. For more information, see [Get started with Sampler.](/docs/guides/get-started-with-primitives#get-started-with-sampler)

In [ ]:
# Plot the result

values = pub_result.data.evs

errors = pub_result.data.stds

# plotting graph
plt.plot(observables_labels, values, "-o")
plt.xlabel("Observables")
plt.ylabel("Values")
plt.show()

<Image src="../docs/images/guides/hello-world/extracted-outputs/87143fcc-0.svg" alt="Output of the previous code cell" />

> **Note:** Cuando ejecuta su programa cuántico en un dispositivo real, su carga de trabajo debe esperar en una cola antes de ejecutarse. Para ahorrar tiempo, puede usar el siguiente código para ejecutar esta pequeña carga de trabajo en [`fake_provider`](../api/qiskit-ibm-runtime/fake-provider) con el modo de prueba local de Qiskit Runtime. Tenga en cuenta que esto solo es posible para un circuito pequeño. Cuando escale en la siguiente sección, necesitará usar un dispositivo real.
> 
>   ```python
> 
>   # Use el siguiente código en su lugar si desea ejecutar en un simulador:
> 
>   from qiskit_ibm_runtime.fake_provider import FakeBelemV2
>   backend = FakeBelemV2()
>   estimator = Estimator(backend)
> 
>   # Convertir a un circuito ISA y observables mapeados al layout.
> 
>   pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
>   isa_circuit = pm.run(qc)
>   mapped_observables = [
>       observable.apply_layout(isa_circuit.layout) for observable in observables
>   ]
> 
>   job = estimator.run([(isa_circuit, mapped_observables)])
>   result = job.result()
> 
>   # Este es el resultado de todo el envío. Envió un Pub,
>   # por lo que contiene un resultado interno (y algunos metadatos propios).
> 
>   job_result = job.result()
> 
>   # Este es el resultado de nuestro único pub, que tenía cinco observables,
>   # por lo que contiene información sobre los cinco.
> 
>   pub_result = job.result()[0]
>   ```
### Paso 4. Analizar los resultados
El paso de análisis es típicamente donde puede realizar un procesamiento posterior de sus resultados usando, por ejemplo, mitigación de errores de medición o extrapolación de ruido cero (ZNE). Puede alimentar estos resultados en otro flujo de trabajo para un análisis adicional o preparar un gráfico de los valores y datos clave. En general, este paso es específico de su problema. Para este ejemplo, grafique cada uno de los valores esperados que se midieron para nuestro circuito.

Los valores esperados y las desviaciones estándar para los observables que especificó a Estimator se acceden a través de los atributos `PubResult.data.evs` y `PubResult.data.stds` del resultado del trabajo. Para obtener los resultados de Sampler, use la función `PubResult.data.meas.get_counts()`, que devolverá un `dict` de mediciones en forma de cadenas de bits como claves y conteos como sus valores correspondientes. Para más información, consulte [Comenzar con Sampler.](/guides/get-started-with-primitives#get-started-with-sampler)

In [8]:
# Make sure the results follow the claim from the previous markdown cell.
# This can happen when the device occasionally behaves strangely. If this cell
# fails, you may just need to run the notebook again.

_results = {obs: val for obs, val in zip(observables_labels, values)}
for _label in ["IZ", "IX", "ZI", "XI"]:
    assert abs(_results[_label]) < 0.2
for _label in ["XX", "ZZ"]:
    assert _results[_label] > 0.8

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/87143fcc-0.svg)

Observe que para los qubits 0 y 1, los valores esperados independientes de X y Z son 0, mientras que las correlaciones (`XX` y `ZZ`) son 1. Esta es una característica del entrelazamiento cuántico.

In [ ]:
def get_qc_for_n_qubit_GHZ_state(n: int) -> QuantumCircuit:
    """This function will create a qiskit.QuantumCircuit (qc) for an n-qubit GHZ state.

    Args:
        n (int): Number of qubits in the n-qubit GHZ state

    Returns:
        QuantumCircuit: Quantum circuit that generate the n-qubit GHZ state, assuming all qubits start in the 0 state
    """
    if isinstance(n, int) and n >= 2:
        qc = QuantumCircuit(n)
        qc.h(0)
        for i in range(n - 1):
            qc.cx(i, i + 1)
    else:
        raise Exception("n is not a valid input")
    return qc


# Create a new circuit with two qubits (first argument) and two classical
# bits (second argument)
n = 100
qc = get_qc_for_n_qubit_GHZ_state(n)

## Escalar a grandes números de qubits
En computación cuántica, el trabajo a escala de utilidad es crucial para avanzar en el campo. Dicho trabajo requiere que los cálculos se realicen a una escala mucho mayor; trabajar con circuitos que podrían usar más de 100 qubits y más de 1000 compuertas. Este ejemplo demuestra cómo puede lograr un trabajo a escala de utilidad en QPUs de IBM&reg; creando y analizando un estado GHZ de 100 qubits. Utiliza el flujo de trabajo de patrones de Qiskit y termina midiendo el valor esperado $\langle Z_0 Z_i \rangle $ para cada qubit.

### Paso 1. Mapear el problema
Escriba una función que devuelva un `QuantumCircuit` que prepare un estado GHZ de $n$ qubits (esencialmente un estado de Bell extendido), luego use esa función para preparar un estado GHZ de 100 qubits y recopilar los observables que se medirán.

In [ ]:
# ZZII...II, ZIZI...II, ... , ZIII...IZ
operator_strings = [
    "Z" + "I" * i + "Z" + "I" * (n - 2 - i) for i in range(n - 1)
]
print(operator_strings)
print(len(operator_strings))

operators = [SparsePauliOp(operator) for operator in operator_strings]

['ZZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

A continuación, mapee a los operadores de interés. Este ejemplo utiliza los operadores `ZZ` entre qubits para examinar el comportamiento a medida que se alejan más. Valores esperados cada vez más inexactos (corrompidos) entre qubits distantes revelarían el nivel de ruido presente.

In [ ]:
service = QiskitRuntimeService()

backend = service.least_busy(
    simulator=False, operational=True, min_num_qubits=100
)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

isa_circuit = pm.run(qc)
isa_operators_list = [op.apply_layout(isa_circuit.layout) for op in operators]

### Step 3. Execute on hardware

Submit the job and enable error suppression by using a technique to reduce errors called [dynamical decoupling.](../api/qiskit-ibm-runtime/options-dynamical-decoupling-options) The resilience level specifies how much resilience to build against errors. Higher levels generate more accurate results, at the expense of longer processing times.  For further explanation of the options set in the following code, see [Configure error mitigation for Qiskit Runtime.](/docs/guides/configure-error-mitigation)

In [ ]:
options = EstimatorOptions()
options.resilience_level = 1
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = "XY4"

# Create an Estimator object
estimator = Estimator(backend, options=options)

In [13]:
# Submit the circuit to Estimator
job = estimator.run([(isa_circuit, isa_operators_list)])
job_id = job.job_id()
print(job_id)

d5k9mmqvcahs73a1ni3g


### Paso 3. Ejecutar en hardware
Envíe el trabajo y habilite la supresión de errores utilizando una técnica para reducir errores llamada [desacoplamiento dinámico.](../api/qiskit-ibm-runtime/options-dynamical-decoupling-options) El nivel de resiliencia especifica cuánta resiliencia construir contra errores. Los niveles más altos generan resultados más precisos, a expensas de tiempos de procesamiento más largos. Para obtener más información sobre las opciones establecidas en el siguiente código, consulte [Configurar la mitigación de errores para Qiskit Runtime.](/guides/configure-error-mitigation)

In [ ]:
# data
data = list(range(1, len(operators) + 1))  # Distance between the Z operators
result = job.result()[0]
values = result.data.evs  # Expectation value at each Z operator.
values = [
    v / values[0] for v in values
]  # Normalize the expectation values to evaluate how they decay with distance.

# plotting graph
plt.plot(data, values, marker="o", label="100-qubit GHZ state")
plt.xlabel("Distance between qubits $i$")
plt.ylabel(r"$\langle Z_i Z_0 \rangle / \langle Z_1 Z_0 \rangle $")
plt.legend()
plt.show()

<Image src="../docs/images/guides/hello-world/extracted-outputs/de91ebd0-0.svg" alt="Output of the previous code cell" />

The previous plot shows that as the distance between qubits increases, the signal decays because of the presence of noise.

## Next steps

<Admonition type="tip" title="Recommendations">
  -   Try one of these tutorials:
      - [Ground-state energy estimation of the Heisenberg chain with VQE](/docs/tutorials/spin-chain-vqe)
      - Solve optimization problems using [QAOA](/docs/tutorials/quantum-approximate-optimization-algorithm)
      - Train [quantum kernel](/docs/tutorials/quantum-kernel-training) models for machine learning tasks
  - Find detailed installation instructions in the [Install Qiskit](/docs/guides/install-qiskit) guide.
  - If you prefer not to install Qiskit locally, read about options to use Qiskit in an [online development environment.](/docs/guides/online-lab-environments)
  - To save multiple account credentials or to specify other account options, see detailed instructions in the [Save your login credentials](/docs/guides/save-credentials#save-your-access-credentials) guide.
</Admonition>